In [1]:
import os
import json
import pandas as pd
from tqdm import tqdm
from ensemble_boxes import *
from glob import glob

# Step1. 환경/경로 설정

In [2]:
WIDTH, HEIGHT = 640, 480 # input resulution이 640 x 480 이라고 가정했습니다. 혹시 아니면 연락주세요!
IMAGE_DIR = "../dataset/images/test_open"
SRC_DIR = "./results"
OUTPUT_FILENAME = "ensenble.txt"

In [3]:
image_files = glob(os.path.join(IMAGE_DIR, "*.png"))
results = glob(os.path.join(SRC_DIR,"*.txt"))
images = [os.path.basename(x).strip('.png') for x in image_files]
assert len(results) == 21 # we have 21 results 
assert len(images) > 0

# Step2. 추론 파일 불러오기 

In [4]:
dataframes = []

for result_file in results:
    assert os.path.isfile(result_file)
    with open(result_file, "r") as f:
        dataframes.append(pd.DataFrame(json.load(f)))
        

# Step3. 앙상블 수행 

In [5]:
dst_list = []
for image_id in tqdm(images):
    image_id = image_id.strip(".png")
    boxes_list = []
    scores_list = []
    labels_list = []
    
    for c_df in dataframes:
        t_df = c_df.loc[c_df.image_id == image_id].reset_index(drop=True)
        t_df['x1y1x2y2'] = t_df['bbox'].apply(lambda x:[
            min(max(x[0] / WIDTH, 0), 1), 
            min(max(x[1] / HEIGHT, 0), 1), 
            min(max((x[0] + x[2]) / WIDTH, 0), 1), 
            min(max((x[1] + x[3]) / HEIGHT, 0), 1), 
        ])
        boxes_list.append(t_df['x1y1x2y2'].to_list())
        scores_list.append(t_df['score'].to_list())
        labels_list.append(t_df['category_id'].to_list())
    
    weights = [1] * len(results)
    iou_thr = 0.8
    skip_box_thr = 0.0001
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    xywh = [[x[0] * WIDTH, x[1] * HEIGHT, (x[2] - x[0]) * WIDTH, (x[3] - x[1]) * HEIGHT] for x in boxes]

    for category_id, bbox, score in zip(labels, xywh, scores):
        dst_list.append({
            'image_id': image_id,
            'category_id': int(category_id),
            'bbox': bbox,
            'score': score
        })
    

100%|██████████████████████████████████████████████████████████████████████████████| 1636/1636 [00:14<00:00, 110.06it/s]


# Step4. 결과 저장

In [7]:
with open(OUTPUT_FILENAME, 'w') as w:
    json.dump(dst_list, w)
print(os.path.abspath(OUTPUT_FILENAME))
assert os.path.isfile(OUTPUT_FILENAME)

/workspace/notebooks/ensenble.txt
